# Clean cho data train và test

In [109]:
import pandas as pd
import io
import re

In [110]:
def clean_hour(hour_str):
    if isinstance(hour_str, str):
        match = re.match(r'(\d{2}:\d{2})', hour_str)
        return match.group(1) if match else hour_str
    return hour_str

def preprocess(df, visibility_fill_value=None):
    # Clean hour
    df['hour'] = df['hour'].apply(clean_hour)
    
    # Parse date
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

    # Parse hour
    df['time'] = pd.to_datetime(df['hour'], format='%H:%M', errors='coerce')
    df['hour_value'] = df['time'].dt.hour
    df['minute'] = df['time'].dt.minute

    # Drop temp columns
    df = df.drop(columns=['date', 'hour', 'time'])

    # Clean visibility
    df['visibility'] = pd.to_numeric(df['visibility'], errors='coerce')

    if visibility_fill_value is not None:
        df['visibility'] = df['visibility'].fillna(visibility_fill_value)
    else:
        df['visibility'] = df['visibility'].fillna(method='ffill')
        # Nếu muốn: df['visibility'] = df['visibility'].interpolate(method='linear')

    return df

def simplify_weather(desc):
    desc = str(desc).lower()
    
    if any(keyword in desc for keyword in ["thunder", "storm", "lightning", "tornado", "hurricane"]):
        return "Stormy"
    
    elif any(keyword in desc for keyword in ["rain", "shower", "drizzle", "sprinkle"]):
        return "Rainy"
    
    elif any(keyword in desc for keyword in ["fog", "haze", "mist", "smoke"]):
        return "Foggy"
    
    elif any(keyword in desc for keyword in ["overcast", "broken clouds", "scattered clouds", "mostly cloudy","more clouds than sun"]):
        return "Cloudy"
    
    elif any(keyword in desc for keyword in ["clear", "sunny", "hot", "warm", "mild", "passing clouds","partly cloudy"]):
        return "Clear"
    
    else:
        return "Unknown"


## 1. Clean dữ liệu train

In [111]:
# === LOAD & CLEAN TRAIN ===
df_train = pd.read_csv("Raw_data/raw_data_train.csv")
df_train.isnull().sum()

date               0
hour               0
temperature        0
weather           29
wind_speed         4
wind_angle         0
humidity           1
pressure          22
visibility     45366
dtype: int64

In [112]:
df_train = df_train.dropna(subset=['weather', 'pressure', 'humidity', 'wind_speed', 'temperature'])
df_train.isnull().sum()

date               0
hour               0
temperature        0
weather            0
wind_speed         0
wind_angle         0
humidity           0
pressure           0
visibility     45357
dtype: int64

In [113]:
df_train = preprocess(df_train)

# Đơn giản hóa mô tả thời tiết
df_train['weather'] = df_train['weather'].apply(simplify_weather)

# Ghi lại giá trị visibility fill từ tập train (dùng giá trị gần nhất không null cuối cùng)
visibility_fill_value = df_train['visibility'].dropna().iloc[-1]

# Lưu dữ liệu đã xử lý
df_train = df_train[['year', 'month', 'day', 'hour_value', 'minute', 'weather', 'temperature', 
                     'wind_speed', 'wind_angle', 'humidity', 'pressure', 'visibility']]
df_train.to_csv("Clean_data/clean_data_train.csv", index=False)
df_train.head()

C:\Users\nptha\AppData\Local\Temp\ipykernel_37188\2670348704.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['visibility'] = df['visibility'].fillna(method='ffill')


,year,month,day,hour_value,minute,weather,temperature,wind_speed,wind_angle,humidity,pressure,visibility
0,2019,1,1,0,0,Clear,19.0,6.0,300.0,88.0,1023.0,6.0
1,2019,1,1,0,30,Clear,19.0,4.0,310.0,88.0,1023.0,6.0
2,2019,1,1,1,0,Clear,19.0,6.0,300.0,88.0,1022.0,6.0
3,2019,1,1,1,30,Clear,19.0,6.0,310.0,88.0,1022.0,6.0
4,2019,1,1,2,0,Clear,19.0,9.0,310.0,88.0,1022.0,6.0


In [114]:

# Áp dụng hàm vào dữ liệu và lọc ra các mô tả chưa được phân loại (Unknown)
df_train['weather_simplified'] = df_train['weather'].apply(simplify_weather)
unknown_weather = df_train[df_train['weather_simplified'] == 'Unknown']['weather'].value_counts()
unknown_weather

weather
Cloudy    11932
Name: count, dtype: int64

## 2. Clean dữ liệu test với tham số đa sử dụng cho train trước đó

In [115]:
# === LOAD & CLEAN TEST ===
df_test = pd.read_csv("Raw_data/raw_data_test.csv")
df_test.isnull().sum()

date               0
hour               0
temperature        0
weather            0
wind_speed         0
wind_angle         0
humidity           0
pressure           1
visibility     17099
dtype: int64

In [116]:
df_test = df_test.dropna(subset=['weather', 'pressure', 'humidity', 'wind_speed', 'temperature'])
df_test.isnull().sum()

date               0
hour               0
temperature        0
weather            0
wind_speed         0
wind_angle         0
humidity           0
pressure           0
visibility     17099
dtype: int64

In [117]:
df_test = preprocess(df_test, visibility_fill_value=visibility_fill_value)
df_test['weather'] = df_test['weather'].apply(simplify_weather)
df_test = df_test[['year', 'month', 'day', 'hour_value', 'minute', 'weather', 'temperature', 
                   'wind_speed', 'wind_angle', 'humidity', 'pressure', 'visibility']]
df_test.head()

,year,month,day,hour_value,minute,weather,temperature,wind_speed,wind_angle,humidity,pressure,visibility
0,2024,1,1,0,0,Foggy,22.0,4.0,320.0,100.0,1017.0,5.0
1,2024,1,1,0,30,Foggy,23.0,2.0,340.0,94.0,1017.0,5.0
2,2024,1,1,1,0,Foggy,22.0,2.0,320.0,100.0,1016.0,5.0
3,2024,1,1,1,30,Foggy,22.0,2.0,0.0,94.0,1016.0,5.0
4,2024,1,1,2,0,Foggy,22.0,4.0,320.0,100.0,1016.0,5.0


In [118]:
df_test.isnull().sum()

year           0
month          0
day            0
hour_value     0
minute         0
weather        0
temperature    0
wind_speed     0
wind_angle     0
humidity       0
pressure       0
visibility     0
dtype: int64

In [119]:
df_test.to_csv("Clean_data/clean_data_test.csv", index=False)

In [90]:
print("✅ Hoàn tất xử lý dữ liệu train và test.")

✅ Hoàn tất xử lý dữ liệu train và test.
